Henter oplysninger på sammenlagte kommuner fra wikipedia


In [383]:
import pandas as pd

# henter tabeller på wikipedia
df = pd.read_html('https://da.wikipedia.org/wiki/Strukturreformen')



In [384]:
# trækker de fem relevante tabeller ud

reg_syd = df[16]
reg_sjæl = df[15]
reg_nordj = df[14]
reg_midtj = df[13]
reg_h = df[12]


In [385]:
#removing multiindex
reg_syd.columns = range(len(reg_syd.columns))
reg_sjæl.columns = range(len(reg_syd.columns))
reg_nordj.columns = range(len(reg_syd.columns))
reg_midtj.columns = range(len(reg_syd.columns))
reg_h.columns = range(len(reg_syd.columns))


In [386]:
# adding region data and merging to one dataframe
reg_syd["region"] = "syd"
reg_sjæl["region"] = "sjæl"
reg_nordj["region"] = "nord"
reg_midtj["region"] = "midt"
reg_h["region"] = "h"
samlet = pd.concat([reg_syd, reg_h, reg_nordj, reg_midtj, reg_sjæl],ignore_index=True)
kolonne_navne = ["Ny_kommune", "Tidligere", "Bemærkninger", "region"]
samlet.columns = kolonne_navne
samlet = samlet.drop(columns = ["Bemærkninger"])

In [387]:
# Manually editing 
samlet.loc[samlet['Ny_kommune'] == 'Haderslev', 'Tidligere'] = "Gram, Haderslev, Vojens"
samlet.loc[samlet['Ny_kommune'] == 'Kolding', 'Tidligere'] = "Christiansfeld, Kolding, Lunderskov, Vamdrup"
samlet.loc[samlet['Ny_kommune'] == 'Tønder', 'Tidligere'] = "Tønder, Bredebro, Højer, Løgumkloster, Nørre Rangstrup, Skærbæk"
samlet.loc[samlet['Ny_kommune'] == 'Vejle', 'Tidligere'] = "Børkop, Egtved, Give, Jelling, Vejle"
samlet.loc[samlet['Ny_kommune'] == 'Frederikssund', 'Tidligere'] = "Frederikssund, Jægerspris, Skibby, Slangerup"
samlet.loc[samlet['Ny_kommune'] == 'Hillerød', 'Tidligere'] = "Hillerød, Skævinge"
samlet.loc[samlet['Ny_kommune'] == 'Skanderborg', 'Tidligere'] = "Galten, Hørning, Ry, Skanderborg"
samlet.loc[samlet['Ny_kommune'] == 'Randers', 'Tidligere'] = "Langå, Nørhald, Purhus, Randers"
samlet.loc[samlet['Ny_kommune'] == 'Norddjurs', 'Tidligere'] = "Grenaa, Nørre Djurs, Rougsø"
samlet.loc[samlet['Ny_kommune'] == 'Horsens', 'Tidligere'] = "Brædstrup, Gedved, Horsens"
samlet.loc[samlet['Ny_kommune'] == 'Vesthimmerland', 'Tidligere'] = "Farsø, Løgstør, Aalestrup, Aars"
samlet.loc[samlet['Ny_kommune'] == 'Mariagerfjord', 'Tidligere'] = "Arden, Hadsund, Hobro, Mariager"
samlet.loc[samlet['Ny_kommune'] == 'Favrskov', 'Tidligere'] = "Hadsten, Hinnerup, Hammel, Hvorslev"
samlet.loc[samlet['Ny_kommune'] == 'Hedensted', 'Tidligere'] = "Hedensted, Juelsminde, Tørring-Uldum"
samlet.loc[samlet['Ny_kommune'] == 'Nordfyn', 'Ny_kommune'] = "Nordfyns"




In [388]:
samlet = samlet.assign(Tidligere=samlet['Tidligere'].str.split(', ')).explode('Tidligere').reset_index(drop=True)
samlet = samlet.rename(columns={"Ny_kommune": 'muni_name', 'Tidligere':'old_muni_name'})

# mere manuelt
samlet.loc[samlet['old_muni_name'] == 'Blaabjerg', 'old_muni_name'] = "Blåbjerg"
samlet.loc[samlet['old_muni_name'] == 'Århus', 'old_muni_name'] = "Århus"
samlet.loc[samlet['old_muni_name'] == 'Aarhus', 'old_muni_name'] = "Århus"
samlet.loc[samlet['old_muni_name'] == 'Nykøbing Falster', 'old_muni_name'] = "Nykøbing-Falster"
samlet.loc[samlet['old_muni_name'] == 'Nørre-Snede', 'old_muni_name'] = "Nørre Snede"



In [390]:
# Henter data til at indsætte muni_codes
FOLK1A = 'https://api.statbank.dk/v1/data/FOLK1A/CSV?valuePresentation=CodeAndValue&OMR%C3%85DE=*&Tid=*'
df = pd.read_csv(FOLK1A, sep = ";")
df['muni_code'] = df['OMRÅDE'].str.extract(r'(\d{3})')   # extracting municipality code
df['muni_name'] = df['OMRÅDE'].str.extract(r'(\d{3}\s+(.*))')[1] # extracting municipalit name
df = df\
    .drop(columns=["OMRÅDE", "TID", "INDHOLD"])\
    .drop_duplicates()

samlet = samlet\
    .merge(df, how='left')

# Henter data til at indsætte gamle muni_codes
BEF1A = "https://api.statbank.dk/v1/data/BEF1A/CSV?valuePresentation=CodeAndValue&OMR%C3%85DE=*&Tid=*"
df = pd.read_csv(BEF1A, sep = ";")
df['old_muni_code'] = df['OMRÅDE'].str.extract(r'(\d{3})')   # extracting municipality code
df['old_muni_name'] = df['OMRÅDE'].str.extract(r'(\d{3}\s+(.*))')[1] # extracting municipalit name
df['old_muni_name'] = df['old_muni_name'].str.replace("(-2005/2006)", "", regex = False)
df['old_muni_name'] = df['old_muni_name'].str.replace("(- 2002)", "", regex = False)



df = df\
    .drop(columns = ["OMRÅDE", 'TID', 'INDHOLD'])\
    .drop_duplicates()
samlet = samlet\
    .merge(df, how = 'left')

# cooercing to integer
samlet['old_muni_code'] = pd.to_numeric(samlet["old_muni_code"], errors='coerce', downcast = 'integer')



In [391]:
samlet.to_csv("nye_og_gamle_kommuner.csv")

,OMRÅDE,old_muni_code,old_muni_name
0,000 Hele landet,000,Hele landet
28,100 København og Frederiksberg,100,København og Frederiksberg
56,101 København,101,København
84,147 Frederiksberg,147,Frederiksberg
112,150 Københavns Amt,150,Københavns Amt
140,165 Albertslund,165,Albertslund
168,151 Ballerup,151,Ballerup
196,153 Brøndby,153,Brøndby
224,155 Dragør,155,Dragør
252,157 Gentofte,157,Gentofte
